In [1]:
import pandas as pd
import requests

In [2]:
import requests


url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"52.31","lon":"13.24","radiusKm":"50","limit":"16","withFlightInfoOnly":"true"}

headers = {
	"x-rapidapi-key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"searchBy":{"lat":52.31,"lon":13.24},"count":2,"items":[{"icao":"EDDB","iata":"BER","name":"Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE","timeZone":"Europe/Berlin"},{"icao":"EDDT","iata":"TXL","name":"Berlin -Tegel","shortName":"-Tegel","municipalityName":"Berlin","location":{"lat":52.5597,"lon":13.287699},"countryCode":"DE","timeZone":"Europe/Berlin"}]}


Let's view the response as `.json()` instead of `.text` so that it's easier to read

In [3]:
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'count': 2,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin'},
  {'icao': 'EDDT',
   'iata': 'TXL',
   'name': 'Berlin -Tegel',
   'shortName': '-Tegel',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.5597, 'lon': 13.287699},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin'}]}

We can now turn this into a dataframe using `.json_normalize()`

In [4]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699


Let's now use this for the latitude and longitude of multiple cities

In [5]:
def get_airports(latitudes, longitudes):
  # API headers
  headers = {
      "X-RapidAPI-Key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b",
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
  }

  querystring = {"withFlightInfoOnly": "true"}

  # DataFrame to store results
  all_airports = []

  for lat, lon in zip(latitudes, longitudes):
    # Construct the URL with the latitude and longitude
    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"

    # Make the API request
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
      data = response.json()
      airports = pd.json_normalize(data.get('items', []))
      all_airports.append(airports)

  return pd.concat(all_airports, ignore_index=True)

In [6]:
# coordinates for Berlin, Paris, London
latitudes = [52.5200, 48.8567, 51.5072]
longitudes = [13.4050, 2.3522, -0.1275]

get_airports(latitudes, longitudes)

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889


** Arrivals information**

In [7]:
def flight_arrival_for_airports(airport_icaos):
  return pd.DataFrame()

In [8]:
! pip install dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import dotenv
dotenv.load_dotenv()

False

In [10]:
import os


In [11]:
db_username = os.getenv('mysql_username')
db_password = os.getenv('mysql_password')
db_host = 'localhost'
db_port = '3306'
db_name = 'sql_workshop'
connection_string = f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
cities_df = pd.read_sql('static_cities', con=connection_string)

RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

In [ ]:
def get_airports(cities_df):
  # API headers
  headers = {
      "X-RapidAPI-Key": os.getenv('AeroDataBoxAPIKey'),
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
  }

  querystring = {"withFlightInfoOnly": "true"}

  # DataFrame to store results
  all_airports = []

  #for lat, lon, name in zip(cities_df['latitude'], cities_df['longitude'], cities_df['name']):
  for i, city in cities_df.iterrows():
    lat = city['latitude']
    lon = city['longitude']
    # Construct the URL with the latitude and longitude
    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"

    # Make the API request
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
      data = response.json()
      airports = pd.json_normalize(data.get('items', []))
      airports['city_id'] = city['id']
      all_airports.append(airports)

  return pd.concat(all_airports, ignore_index=True)

In [ ]:
airports_df=get_airports(cities_df)

In [ ]:
airports_df[['icao','city_id','name','location.lat','location.lon']]\
    .rename(columns={'location.lat':'latitude','location.lon':'longitude'})\
    .to_sql('airports', con=connection_string, if_exists='append', index=False)

4

In [ ]:
airports_df = pd.read_sql('airports', con=connection_string)


In [ ]:
for a,b in timeslots:
    print(a,b)

00:00 11:59
12:00 23:59


In [ ]:
def get_flights_info(airports_df):
    from datetime import datetime, timedelta
    tomorrow=(datetime.now()+timedelta(days=1)).strftime('%Y-%m-%d')
    tomorrow
    import requests
    timeslots = [('00:00','11:59'),('12:00','23:59')]
    list_of_flights = []
    for i, airport in airports_df.iterrows():
        for start, end in timeslots:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport['icao']}/{tomorrow}T{start}/{tomorrow}T{end}"

            querystring = {"withLeg":"false","direction":"Both","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

            headers = {
                "x-rapidapi-key": os.environ.get('AeroDataBoxAPIKey'),
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.get(url, headers=headers, params=querystring)

            if response.status_code != 200:
                print(response.status_code, airport['icao'])
                continue
            flights = pd.json_normalize(response.json().get('arrivals', []))
            flights['airport_icao'] = airport['icao']
            list_of_flights.append(flights)
    return pd.concat(list_of_flights, ignore_index=True)


In [ ]:
flights_df = get_flights_info(airports_df)

204 EDDT
204 EDDT


In [ ]:
flights_df['movement.revisedTime.local'] = flights_df['movement.revisedTime.local'].str[:-6]

In [ ]:
flights_df

,number,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.airport.timeZone,movement.scheduledTime.utc,movement.scheduledTime.local,...,movement.baggageBelt,movement.quality,aircraft.model,airline.name,airline.iata,airline.icao,callSign,aircraft.reg,aircraft.modeS,airport_icao
0,HU 489,Expected,IsOperator,False,ZBAA,PEK,Beijing,Asia/Shanghai,2025-03-27 05:40Z,2025-03-27 06:40+01:00,...,A3,"[Basic, Live]",Airbus A330-300,Hainan,HU,CHH,NaN,NaN,NaN,EDDB
1,QR 79,Expected,IsOperator,False,OTHH,DOH,Doha,Asia/Qatar,2025-03-27 05:55Z,2025-03-27 06:55+01:00,...,A3,"[Basic, Live]",Boeing 787-9,Qatar,QR,QTR,NaN,NaN,NaN,EDDB
2,FR 9,Expected,IsOperator,False,LIMC,MXP,Milan,Europe/Rome,2025-03-27 06:30Z,2025-03-27 07:30+01:00,...,C2,"[Basic, Live]",Airbus A320,Ryanair,FR,RYR,RYR9,9H-LOU,4D22B0,EDDB
3,FR 1149,Expected,IsOperator,False,LBSF,SOF,Sofia,Europe/Sofia,2025-03-27 06:30Z,2025-03-27 07:30+01:00,...,C1,"[Basic, Live]",Boeing 737-800 (winglets),Ryanair,FR,RYR,NaN,NaN,NaN,EDDB
4,EW 2000,Expected,IsOperator,False,EDDS,STR,Stuttgart,Europe/Berlin,2025-03-27 06:35Z,2025-03-27 07:35+01:00,...,A3,"[Basic, Live]",Airbus A319,Eurowings,EW,EWG,NaN,NaN,NaN,EDDB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,AF 1122,Expected,IsOperator,False,LFPG,CDG,Paris,Europe/Paris,2025-03-27 21:20Z,2025-03-27 22:20+01:00,...,NaN,"[Basic, Live]",BCS3,Air France,AF,AFR,NaN,NaN,NaN,EDDM
828,BA 958,Expected,IsOperator,False,EGLL,LHR,London,Europe/London,2025-03-27 21:45Z,2025-03-27 22:45+01:00,...,NaN,"[Basic, Live]",Airbus A320,British,BA,BAW,NaN,NaN,NaN,EDDM
829,IB 747,Expected,IsOperator,False,LEMD,MAD,Madrid,Europe/Madrid,2025-03-27 21:45Z,2025-03-27 22:45+01:00,...,NaN,"[Basic, Live]",Airbus A319,Iberia,IB,IBE,NaN,NaN,NaN,EDDM
830,LH 2483,Expected,IsOperator,False,EGLL,LHR,London,Europe/London,2025-03-27 22:05Z,2025-03-27 23:05+01:00,...,NaN,"[Basic, Live]",Airbus A320 NEO,Lufthansa,LH,DLH,NaN,NaN,NaN,EDDM
